In [1]:
include("../src/ProcessSequence.jl")

Main.ProcessSequence

In [7]:
project_dir = dirname(@__DIR__)

using JSON: parse
project_json = parse(read(joinpath(project_dir, "project.json"), String))

sample_name = project_json["sample_name"]

output_dir = joinpath(project_dir, "output")
input_dir = joinpath(project_dir, "input")
sample_dir = joinpath(input_dir, sample_name)
reference_dir = joinpath(input_dir, "reference")
snpeff_jar = "/opt/snpeff/snpEff/snpEff.jar"

"/opt/snpeff/snpEff/snpEff.jar"

In [8]:
if !isdir(reference_dir)

    ProcessSequence.print_and_run_cmd(`unzip -o -d $input_dir $reference_dir.zip`)
    
end

In [9]:
if !isfile(snpeff_jar)
    
    throw("$snpeff is missing.")
    
end

In [10]:
ProcessSequence.check_program()

Checking program...
`which skewer`
/opt/conda/bin/skewer
`which fastqc`
/opt/conda/bin/fastqc
`which bgzip`
/opt/conda/bin/bgzip
`which tabix`
/opt/conda/bin/tabix
`which minimap2`
/opt/conda/bin/minimap2
`which samtools`
/opt/conda/bin/samtools
`which bcftools`
/opt/conda/bin/bcftools
`which kallisto`
/opt/conda/bin/kallisto
`bash -c 'source activate py2 && which configManta.py'`
/opt/conda/envs/py2/bin/configManta.py
`bash -c 'source activate py2 && which configureStrelkaGermlineWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaGermlineWorkflow.py
`bash -c 'source activate py2 && which configureStrelkaSomaticWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaSomaticWorkflow.py


In [12]:
process_dna_arguments = (
    joinpath(
        reference_dir,
        "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz",
    ),
    joinpath(reference_dir, "chromosome.bed.gz"),
    joinpath(reference_dir, "chrn_n.tsv"),
    project_json["n_job"],
    project_json["gb_memory"],
    5,
    snpeff_jar,
)

("/home/jovyan/ProcessSequence.jl/input/reference/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz", "/home/jovyan/ProcessSequence.jl/input/reference/chromosome.bed.gz", "/home/jovyan/ProcessSequence.jl/input/reference/chrn_n.tsv", 8, 40, 5, "/opt/snpeff/snpEff/snpEff.jar")

In [15]:
if all((
    in(key, keys(project_json)) for key in ("germ_dna.1.fastq.gz", "germ_dna.2.fastq.gz")
))

    ProcessSequence.process_germ_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, string("process_germ_dna_", sample_name)),
        process_dna_arguments...,
    )

end

(2020-12-06T07:25:30.241) Trimming sequence ...
`skewer --threads 8 -x AGATCGGAAGAGC --end-quality 20 --mode pe --compress --output /home/jovyan/ProcessSequence.jl/output/process_germ_dna_test_high_quality/trim_sequence/germ --quiet /home/jovyan/ProcessSequence.jl/input/test_high_quality/germ_dna.1.1m.fastq.gz /home/jovyan/ProcessSequence.jl/input/test_high_quality/germ_dna.2.1m.fastq.gz`
.--. .-.
: .--': :.-.
`. `. : `'.' .--. .-..-..-. .--. .--.
_`, :: . `.' '_.': `; `; :' '_.': ..'
`.__.':_;:_;`.__.'`.__.__.'`.__.':_;
skewer v0.2.2 [April 4, 2016]
Parameters used:
-- 3' end adapter sequence (-x):	AGATCGGAAGAGC
-- maximum error ratio allowed (-r):	0.100
-- maximum indel error ratio allowed (-d):	0.030
-- end quality threshold (-q):		20
-- minimum read length allowed after trimming (-l):	18
-- file format (-f):		Sanger/Illumina 1.8+ FASTQ (auto detected)
-- number of concurrent threads (-t):	8
Sun Dec  6 07:25:30 2020 >> started

Sun Dec  6 07:26:00 2020 >> done (30.602s)
1000000 read

Started analysis of germ-trimmed-pair1.fastq.gz
Started analysis of germ-trimmed-pair2.fastq.gz
Approx 5% complete for germ-trimmed-pair1.fastq.gz
Approx 5% complete for germ-trimmed-pair2.fastq.gz
Approx 10% complete for germ-trimmed-pair1.fastq.gz
Approx 10% complete for germ-trimmed-pair2.fastq.gz
Approx 15% complete for germ-trimmed-pair1.fastq.gz
Approx 15% complete for germ-trimmed-pair2.fastq.gz
Approx 20% complete for germ-trimmed-pair1.fastq.gz
Approx 20% complete for germ-trimmed-pair2.fastq.gz
Approx 25% complete for germ-trimmed-pair1.fastq.gz
Approx 25% complete for germ-trimmed-pair2.fastq.gz
Approx 30% complete for germ-trimmed-pair1.fastq.gz
Approx 30% complete for germ-trimmed-pair2.fastq.gz
Approx 35% complete for germ-trimmed-pair1.fastq.gz
Approx 35% complete for germ-trimmed-pair2.fastq.gz
Approx 40% complete for germ-trimmed-pair1.fastq.gz
Approx 40% complete for germ-trimmed-pair2.fastq.gz
Approx 45% complete for germ-trimmed-pair1.fastq.gz
Approx 50% complete fo

LoadError: failed process: Process(`java -Xmx40g -jar /opt/snpeff/snpEff/snpEff.jar GRCh38.99 -noLog -verbose -csvStats /home/jovyan/ProcessSequence.jl/output/process_germ_dna_test_high_quality/find_variant/snpeff/stats.csv -htmlStats /home/jovyan/ProcessSequence.jl/output/process_germ_dna_test_high_quality/find_variant/snpeff/stats.html /home/jovyan/ProcessSequence.jl/output/process_germ_dna_test_high_quality/find_variant/concat.vcf.gz`, ProcessExited(255)) [255]


In [10]:
if all((
    in(key, keys(project_json))
    for
    key in (
        "germ_dna.1.fastq.gz",
        "germ_dna.2.fastq.gz",
        "soma_dna.1.fastq.gz",
        "soma_dna.2.fastq.gz",
    )
))

    ProcessSequence.process_soma_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, "process_soma_dna"),
        process_dna_arguments...,
    )

end

In [ ]:
if all((
    in(key, keys(project_json)) for key in ("soma_rna.1.fastq.gz", "soma_rna.2.fastq.gz")
))

    ProcessSequence.process_soma_rna(
        joinpath(project_dir, project_json["soma_rna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_rna.2.fastq.gz"]),
        joinpath(output_dir, "process_soma_rna"),
        joinpath(reference_dir, "Homo_sapiens.GRCh38.cdna.all.fa.gz"),
        project_json["n_job"],
    )

end